# Lap Time Prediction Models

This notebook demonstrates training and using ML models for lap-by-lap predictions during a race.

In [ ]:
# Force reload modules
import importlib
import sys

modules_to_reload = [mod for mod in sys.modules.keys() if 'toyota_gr_cup_analytics' in mod]
for mod in modules_to_reload:
    del sys.modules[mod]

print(f"Removed {len(modules_to_reload)} modules from cache")

## Load and Prepare Data

In [ ]:
from toyota_gr_cup_analytics.data_processing.loaders import load_lap_data, load_weather_data
from toyota_gr_cup_analytics.models.feature_engineering import engineer_features

# Load data
lap_data = load_lap_data("barber", 1)
weather_data = load_weather_data("barber", 1)

print(f"Loaded {len(lap_data)} lap records")
print(f"Loaded {len(weather_data)} weather records")

In [ ]:
# Engineer features
X, y = engineer_features(lap_data, weather_data)

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns: {list(X.columns)}")
print(f"\nTarget statistics:")
print(f"  Mean lap time: {y.mean():.3f}s")
print(f"  Std lap time: {y.std():.3f}s")
print(f"  Min lap time: {y.min():.3f}s")
print(f"  Max lap time: {y.max():.3f}s")

## Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split 80/20 train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

## Train Tire Degradation Model

In [ ]:
from toyota_gr_cup_analytics.models.predictive import TireDegradationModel

# Train linear degradation model
tire_model = TireDegradationModel(degradation_type="linear")
tire_model.fit(X_train, y_train)

# Evaluate
tire_metrics = tire_model.evaluate(X_test, y_test)
print("\nTire Degradation Model Performance:")
for metric, value in tire_metrics.items():
    print(f"  {metric.upper()}: {value:.3f}")

# Show coefficients if linear
if tire_model.coefficients:
    print("\nModel Coefficients (seconds per unit):")
    for feature, coef in tire_model.coefficients.items():
        print(f"  {feature}: {coef:.4f}")

## Train Lap Time Prediction Model

In [ ]:
from toyota_gr_cup_analytics.models.predictive import LapTimePredictorModel

# Train Random Forest model
lap_time_model = LapTimePredictorModel(
    n_estimators=100,
    model_type="random_forest",
    random_state=42
)
lap_time_model.fit(X_train, y_train)

# Evaluate
lap_time_metrics = lap_time_model.evaluate(X_test, y_test)
print("\nLap Time Prediction Model Performance:")
for metric, value in lap_time_metrics.items():
    print(f"  {metric.upper()}: {value:.3f}")

# Feature importance
print("\nTop 10 Important Features:")
importances = lap_time_model.feature_importance()
sorted_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)[:10]
for feature, importance in sorted_features:
    print(f"  {feature}: {importance:.4f}")

## Predict Stint Performance

In [ ]:
# Predict a 15-lap stint for car #2
base_lap_time = y_train.min() + 1.0  # Slightly slower than fastest
stint_length = 15
car_id = 2

predicted_times, stint_stats = tire_model.predict_stint_performance(
    base_lap_time=base_lap_time,
    stint_length=stint_length,
    car_id=car_id,
    start_lap=10,
    temperature=32.0
)

print(f"\nStint Prediction for Car #{car_id} (15 laps):")
print(f"  Fastest lap: {stint_stats['fastest_lap']:.3f}s")
print(f"  Slowest lap: {stint_stats['slowest_lap']:.3f}s")
print(f"  Average lap: {stint_stats['average_lap']:.3f}s")
print(f"  Total degradation: {stint_stats['total_degradation']:.3f}s")
print(f"  Degradation per lap: {stint_stats['degradation_per_lap']:.3f}s/lap")

print("\nLap-by-lap predictions:")
for i, lap_time in enumerate(predicted_times, 1):
    print(f"  Lap {i}: {lap_time:.3f}s")

## Real-Time Prediction Example

In [ ]:
# Simulate predicting the next lap during a race
current_state = {
    'car_id': 5,
    'lap_number': 20,
    'tire_age': 8,
    'stint_number': 2,
    'degradation_rate': 0.05,
    'fuel_load_proxy': 0.5,
    'rolling_mean_3': 99.5,
    'rolling_std_3': 0.3
}

weather_conditions = {
    'air_temp': 26.0,
    'track_temp': 33.0,
    'humidity': 45.0
}

next_lap_prediction, metadata = lap_time_model.predict_next_lap(
    current_state, weather_conditions
)

print(f"\nNext Lap Prediction for Car #{current_state['car_id']}:")
print(f"  Lap #{current_state['lap_number'] + 1}: {next_lap_prediction:.3f}s")
print(f"  Tire age: {metadata['tire_age']} laps")
print(f"  Conditions: {weather_conditions['track_temp']:.1f}°C track, {weather_conditions['air_temp']:.1f}°C air")

## Visualize Predictions vs Actual

In [ ]:
import plotly.graph_objects as go

# Get predictions for test set
y_pred = lap_time_model.predict(X_test)

# Create scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred,
    mode='markers',
    name='Predictions',
    marker=dict(size=4, opacity=0.6)
))

# Add perfect prediction line
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    name='Perfect Prediction',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title="Predicted vs Actual Lap Times",
    xaxis_title="Actual Lap Time (seconds)",
    yaxis_title="Predicted Lap Time (seconds)",
    template="plotly_white",
    height=600
)

fig.show()

## Model Summary

The models are now trained and ready for lap-by-lap predictions during a race. Key capabilities:

1. **Tire Degradation Model**: Predicts how lap times degrade over a stint
2. **Lap Time Predictor**: Predicts next lap time based on current conditions
3. **Feature Importance**: Identifies what factors matter most
4. **Real-time Prediction**: Can update predictions lap-by-lap as race progresses

Next steps:
- Train on multiple races/tracks for better generalization
- Add traffic and position-based features
- Integrate with live timing data feed
- Build strategy optimization on top of predictions